In [1]:
# run in docker:
# sudo docker run -p 8890:8888 -v /home/tehdog/data/dev/2019/pic2kcal-cv-praktikum:/tf/notebooks --runtime=nvidia -it --rm tensorflow/tensorflow:1.12.0-gpu-py3 jupyter notebook --allow-root --notebook-dir=/tf/notebooks
# crashes in TF1.4!
# does not work with Arch Linux TF installation!

In [2]:
%cd ..

/tf/notebooks


In [3]:
!pip install tensorflow-hub
!pip install sentencepiece
!pip install tf-sentencepiece
!pip install tqdm

You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tf_sentencepiece

# Set up graph.
g = tf.Graph()
with g.as_default():
  text_input = tf.placeholder(dtype=tf.string, shape=[None])
  en_de_embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-xling/en-de/1")
  embedded_text = en_de_embed(text_input)
  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

# Initialize session.
session = tf.Session(graph=g)
session.run(init_op)


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0713 20:20:14.771718 140330910422784 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [5]:
from pathlib import Path
import json
import re
import random
import heapq
from operator import itemgetter
from tqdm import tqdm_notebook as tqdm
from math import ceil

data_dir = Path("data")

In [6]:
# jq '.[]' fddb_data_v3.json > fddb_data_v3.jsonl
# jq 'select(([.Bilder[]|select(.title != "Noch kein Foto vorhanden.")]|length) > 0)' fddb_data_v3.jsonl | jq -s > fddb_data_v3_withimg.json
with open(str(data_dir / "fddb_data_v4_withimg.json"), encoding='utf-8') as f:
    fddb = json.load(f)
    # todo: make unique here
    _out_names = [e["name"] for e in fddb]

# jq '.ingredients[]|select(.ingredient)|.ingredient' processed_data.jsonl | jq -s unique > ingredients.json
with open(str(data_dir / "recipes/ingredients.json"), encoding='utf-8') as f:
    _in_names = json.load(f)


In [7]:
def normalize_ingredient(ing: str):
    # ing = re.sub(r"\(([^)])\)", "\g<1>", ing)  # remove stuff in parens
    ing = re.sub(r"\([^)]+\)", "", ing)  # remove stuff in parens
    ing = re.sub(r"(\d+,)?\d+ k?g\b", "", ing)  # remove xyz gram
    ing = re.sub(r",.*", "", ing) # TODO: DONT DO THIS! SIGNIFICANT INFORMATION LOSS
    ing = re.sub(r"\bzum .*", "", ing)
    ing = re.sub(r"\boder\b.*", "", ing)
    ing = ing.strip()
    return ing

from extract_ingredients.util import normalize_out_ingredient
    

In [8]:
in_names = list({normalize_ingredient(ing) for ing in _in_names})
print(len(_in_names))
in_names.sort()
len(in_names)

258944


21410

In [9]:
print(len(_out_names))
out_names = list({ning for ing in _out_names for ning in normalize_out_ingredient(ing)})
out_names.sort()
len(out_names)

122977


147431

In [10]:
#def Matcher():
#    def __init__(self, data_left, data_right, preproc_left, preproc_right):
#        pre_left = list(map(preproc_left, data_left))
#        self.left_vecs = list(zip(data_left, pre_left, get_sentence_vectors(pre_left)))
#        
#        pre_right = list(map(preproc_right, data_right))
#        self.right_vecs = list(zip(data_right, pre_right, get_sentence_vectors(pre_right)))

In [11]:
def make_chunks(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

In [12]:
def get_sentence_vectors(texts):
    bs = 10000
    ccount = len(texts)//bs
    chunks = make_chunks(texts, bs)
    if ccount >= 3:
        chunks = tqdm(chunks, total=ccount)
    for chunk in chunks:
        yield from session.run(embedded_text, feed_dict={text_input: chunk})

In [13]:
def get_match(search: np.array, out_vecs, limit=30):
    it = ((v[0], np.dot(v[1], search)) for v in out_vecs)
    res_list = heapq.nlargest(limit, it, key=itemgetter(1))
    return res_list

In [14]:
out_vecs = list(zip(out_names, get_sentence_vectors(out_names)))

In [15]:
out_vecs[0][1].shape

(512,)

In [16]:
in_names[0], out_names[0]

('8 Kräuter', '"DIE FEINE" Geflügel-Fleischwurst')

In [17]:
in_vecs = list(zip(in_names, get_sentence_vectors(in_names)))

In [18]:
for ingredient, vec in random.sample(in_vecs, 100):
    res_list = get_match(vec, out_vecs)
    print("{} -> {}".format(ingredient, res_list[0]))

Zitrone  mit Schale -> ('Heiße Zitrone', 0.8756706)
Butterkeks mit Schokolade -> ('Schokoladen Butterkeks', 0.9341165)
Lamm - Karree -> ('Kohl - Roulade', 0.8137923)
Kuvertüre und Liebesperlen -> ('Liebesperlen', 0.8599304)
Blüten von der Ringelblume -> ('Ringelblumenblüten', 0.882388)
Grapefruit "Sweetie" -> ('Pink Grapefruit-Apfel-Sweetie', 0.8001827)
Fett für die Hände -> ('Körper Balance', 0.7117213)
Hähnchen – -> ('Hähnchen', 0.7442771)
Gelierpulver ohne Zucker z. B. Konfige -> ('Zucrinet Pulver, Süss, Zuckerersatz', 0.82039046)
Frühlingszwiebel in feine Ringe geschnitten -> ('Brotaufstrich, Frühlingszwiebel-Schinken', 0.86105955)
Crème fraîche mit Käuter -> ('Crème fraîche', 0.87767243)
Streusüße -> ('Streusüße', 1.0000001)
Vollrohrzucker -> ('Vollrohrzucker', 1.0)
Baguette vom Vortag -> ('Bistro Baguette', 0.82806385)
Schokolade 70% -> ('Schokolade 70%', 1.0)
Sauce und Dips nach Geschmack -> ('Süß-Sauer Sauce', 0.77727664)
Bratenfond -> ('Bratenspeck', 0.8664383)
Chilipulver von

In [19]:
z = [inv for inv in in_vecs if inv[0] == 'Kartoffeln'][0]
get_match(z[1], out_vecs)

[('Kartoffeln', 1.0),
 ('Kartoffeln ', 1.0),
 ('Kartoffeln ganz', 0.8977796),
 ('Kartoffeln gekocht', 0.8963243),
 ('Kartoffelstock', 0.8959534),
 ('Kartoffeln, gegart', 0.8914453),
 ('Kartoffeln roh', 0.8905239),
 ('Kartoffeln gebacken', 0.8904908),
 ('Kartoffelscheiben', 0.8901619),
 ('Kartoffeln, ganz', 0.88885343),
 ('Kartoffeln suppe', 0.8838953),
 ('Ganze Kartoffeln', 0.88332707),
 ('Ganze Kartoffeln ', 0.88332707),
 ('Brat Kartoffeln ', 0.88277364),
 ('Kartoffel', 0.88085693),
 ('Kartoffelnudeln', 0.8792579),
 ('Kartoffelsticks', 0.87786144),
 ('Kartoffelsnack', 0.8732353),
 ('Gekochte Kartoffeln', 0.8718474),
 ('Kartoffelpree ', 0.87160146),
 ('Kartoffeln, roh', 0.8711822),
 ('geschälte Kartoffeln', 0.8708629),
 ('Kartoffel Sticks', 0.8684819),
 ('Kartoffeln mehlig', 0.86703193),
 ('Kartoffelklöße', 0.8620798),
 ('Kartoffelpürree', 0.86069757),
 ('Kartoffelpüree', 0.8605306),
 ('Kartoffelpüree ', 0.8605306),
 ('Kartoffel-Gemüse', 0.8604079),
 ('Kartoffelpürre ', 0.85896957)]

In [20]:
z = list(get_sentence_vectors(["Schalotten"]))[0]
get_match(z, out_vecs, limit=30)

[('Schalotten', 1.0000001),
 ('Sülze', 0.89274013),
 ('Ringlotten', 0.8788649),
 ('Schaschlik', 0.8767235),
 ('Selters', 0.87629676),
 ('Selchspeck', 0.87524664),
 ('Leberkse', 0.87103057),
 ('Hustelinchen', 0.86819935),
 ('Schwartenmagen', 0.86798453),
 ('Karlchen', 0.8628453),
 ('Maiskölbchen', 0.86279553),
 ('Grützwust', 0.8598486),
 ('Mürbchen ', 0.85951704),
 ('Maisecke', 0.85870075),
 ('Kersen', 0.85840064),
 ('Käsknacker', 0.858381),
 ('Cemen', 0.8576199),
 ('Lattich', 0.8575938),
 ('Hhnerfilet', 0.8569063),
 ('Ketschup', 0.85674757),
 ('Korneck', 0.85655665),
 ('Körnereck', 0.8554502),
 ('Krönchen', 0.85468316),
 ('Knoferl', 0.85430396),
 ('Landschinken', 0.85422593),
 ('Landschinken ', 0.85422593),
 ('Wicküler ', 0.85283446),
 ('Roggenbrtchen ', 0.8517979),
 ('Salchichn', 0.8514553),
 ('Rohschinken', 0.8509237)]

In [21]:
z = list(get_sentence_vectors([normalize_ingredient("Salz")]))[0]
get_match(z, out_vecs, limit=30)

[('Salz', 1.0),
 ('Salzgurken', 0.9089097),
 ('Gut Salzig', 0.90342623),
 ('Salzbrezeln', 0.8880923),
 ('Salzgurken, salzig', 0.8686831),
 ('Salzmandeln', 0.8676863),
 ('Salzgurke', 0.86386824),
 ('Salzbrezeln, Salzig', 0.8605131),
 ('Salzbrezel', 0.8582727),
 ('Salz-Mandeln', 0.85328484),
 ('salzbrezel', 0.84165347),
 ('Salz, Jodsalz', 0.8251066),
 ('Salz-Diamanten', 0.82034075),
 ('Salzbutter', 0.81924367),
 ('Tafelsalz', 0.8182125),
 ('Salzdillgurken', 0.8158475),
 ('Jod Salz', 0.81424797),
 ('Salz flutes, Salz', 0.8119249),
 ('Meersalz', 0.811814),
 ('Blutdruck Salz', 0.8090307),
 ('Salz-Stangen', 0.80218107),
 ('smorbar gesalzen', 0.7952043),
 ('Chips Salz', 0.79345155),
 ('Salzstangerl', 0.7885661),
 ('Saltufo', 0.78672385),
 ('Salzstange ', 0.7866403),
 ('Salz Kräcker', 0.7803569),
 ('Corn, salz', 0.7801466),
 ('Salz Stangen', 0.7790837),
 ('Koblauch Salz', 0.7786817)]

In [22]:
z = list(get_sentence_vectors([normalize_ingredient("Möhre")]))[0]
get_match(z, out_vecs, limit=30)

[('Möhre', 1.0),
 ('Mürbchen ', 0.86611366),
 ('Maisecke', 0.8569765),
 ('Mairübchen', 0.8499796),
 ('Cemen', 0.84976923),
 ('Hirsegrieß', 0.8468805),
 ('Hirsebllchen', 0.84569955),
 ('Korneck', 0.8455517),
 ('Schmelzkse', 0.83843553),
 ('Hirsebrei', 0.8378884),
 ('Hirse', 0.83783054),
 ('Möhrchen', 0.8373093),
 ('Kareespeck', 0.837133),
 ('Grützwust', 0.8362565),
 ('Hirse Griess', 0.83579063),
 ('Bergpfirsich', 0.83537793),
 ('Kümmelpärle', 0.83503175),
 ('Schalotten', 0.83413565),
 ('Kräcker', 0.83367187),
 ('Gutfried', 0.83342147),
 ('Körnereck', 0.8333955),
 ('Tafelkren', 0.8322475),
 ('Süssmais', 0.8322011),
 ('Karreespeck', 0.83199644),
 ('Karreespeck ', 0.83199644),
 ('Edelschinlen', 0.8316824),
 ('Rotbäckchen', 0.83162296),
 ('Schokostrudel', 0.83152306),
 ('Schokolinsen', 0.83025885),
 ('Krönchen', 0.83021057)]

In [23]:
z = list(get_sentence_vectors([normalize_ingredient("Zitronen")]))[0]
get_match(z, out_vecs, limit=30)

[('Zitronen Saft', 0.93052155),
 ('Zitronenrolle', 0.92803085),
 ('Zitronenwasser', 0.92116797),
 ('Zitronenschale', 0.9154556),
 ('Zitronencreme', 0.91466904),
 ('Zitronensaft', 0.91464496),
 ('Zitronensaft ', 0.91464496),
 ('Zitronenschaum', 0.91345096),
 ('Zitronen-Pesto', 0.906561),
 ('Zitronen limonade', 0.9027358),
 ('Zitronen T', 0.8955948),
 ('Zitronenmelisse', 0.8908546),
 ('Lemon', 0.8900281),
 ('Zitrone', 0.8861021),
 ('Zitronenlimonde', 0.88575715),
 ('Zitronenlimonade', 0.8841475),
 ('Zitronen-Häppchen', 0.87951857),
 ('Zitronensauce', 0.874436),
 ('Zitronen Aroma', 0.8718134),
 ('Zitronen Limonade', 0.8700224),
 ('Zitronen Limonade ', 0.8700224),
 ('Zitronet', 0.8680041),
 ('Zitronen T, Zitrone', 0.86615825),
 ('Zitronenzucker', 0.8657205),
 ('Zitronenwasser, Zitrone', 0.86054575),
 ('Zitronenkuchen', 0.8587698),
 ('Zitronen Kuchen', 0.8571212),
 ('Zitronenschalen', 0.8568182),
 ('Zitronenschalen ', 0.8568182),
 ('Zitronen-Rolle', 0.8536694)]

In [24]:
z = list(get_sentence_vectors([normalize_ingredient("Pfeffer")]))[0]
get_match(z, out_vecs, limit=30)

[('Pfeffer', 1.0),
 ('Pfefferonen', 0.89296865),
 ('Pfefferminz', 0.8850076),
 ('Pfefferbreze', 0.8844123),
 ('Pfeffer Rolle', 0.8822749),
 ('Pfefferblock', 0.88045084),
 ('Pfeffer-Schinken', 0.8785044),
 ('Pfefferminze', 0.8713012),
 ('Pfeffer-Braten', 0.8676797),
 ('Pfefferbeiser', 0.8666811),
 ('Pfeffer-Beißer', 0.866335),
 ('Pfefferminztee', 0.8620248),
 ('Pfefferbeisser', 0.8613457),
 ('Pfeffersäckchen', 0.86078715),
 ('Pfefferbraten', 0.8570888),
 ('Bunter Pfeffer', 0.8531077),
 ('Pfeffer, weiß', 0.84696233),
 ('Pfefferkarree', 0.8463141),
 ('Pfeffer Braten', 0.8403899),
 ('Pfeffernüsse', 0.83831143),
 ('Pfeffer, schwarz', 0.83563584),
 ('Pfefferoni', 0.8356279),
 ('Pfefferbeißer', 0.8338611),
 ('Grüner Pfeffer', 0.8303262),
 ('Pfeffer Soße', 0.8280723),
 ('Pfefferonen, scharf', 0.824779),
 ('Paprikaschote', 0.82357407),
 ('Pfefferschinken', 0.8235053),
 ('Käse, Pfeffer', 0.81792927),
 ('Salz und Pfeffer', 0.81601393)]

In [25]:
z = list(get_sentence_vectors([normalize_ingredient("Zucchini")]))[0]
get_match(z, out_vecs, limit=30)

[('Zucchini', 1.0),
 ('Zucchini ', 1.0),
 ('Zuchini', 0.96570617),
 ('Zucchini-Puffer', 0.95276517),
 ('Zucchini-Gratin', 0.9444525),
 ('Zucchini Pfanne', 0.9339043),
 ('Zucchini Quiche', 0.92734474),
 ('Zucchini, gegart', 0.9196474),
 ('Zucchini-Pfanne', 0.91940373),
 ('Zucchini-Reistopf', 0.9190987),
 ('Zucchini-Fritata', 0.91898525),
 ('Zucchinipuffer', 0.91596097),
 ('Zucchini gebraten', 0.907671),
 ('Gebackene Zucchini', 0.90149117),
 ('Zucchini Püree', 0.89597404),
 ('Currychini', 0.89119816),
 ('Zucchini-puffer', 0.88802767),
 ('Zucchinibrot', 0.88706577),
 ('Zucchini Auflauf', 0.8870287),
 ('Zucchini Cannelloni', 0.8869643),
 ('Papucchini', 0.88294375),
 ('Zucchini-Cheddar-Puffer', 0.8814348),
 ('Zucchini Mischung', 0.8783588),
 ('Crocchini', 0.87423605),
 ('Zuchini gegrillt', 0.8742033),
 ('Zucchini-Paprika-Ragu', 0.8666968),
 ('Gegrillte Zucchini', 0.86394763),
 ('Zucchini Brownie', 0.8617481),
 ('Zucchini-Gemüse-Pfanne', 0.8587595),
 ('Zucchini-Spaghetti', 0.85744786)]

In [26]:
z = list(get_sentence_vectors([normalize_ingredient("Paprikaschote")]))[0]
get_match(z, out_vecs, limit=30)

[('Paprikaschote', 1.0),
 ('Paprikaschoten', 0.98521674),
 ('Paprikagulasch', 0.95074636),
 ('Paprikasülze', 0.9479988),
 ('Paprika-Gulasch', 0.9312785),
 ('Paprikapastete', 0.9309404),
 ('Paprikastreifen', 0.9304859),
 ('Gefüllte Paprikaschote', 0.9249151),
 ('Paprikapulver', 0.92458856),
 ('Paprikacreme', 0.92321056),
 ('Paprikagemüse', 0.9231436),
 ('Paprikaknacker', 0.9185246),
 ('Paprikasalat', 0.9125063),
 ('Paprika Edelsüss', 0.9123241),
 ('Rote Paprikaschoten', 0.9121383),
 ('Paprikamark', 0.91010225),
 ('Paprikareis', 0.90830344),
 ('Paprika-Mandel-Dip', 0.907791),
 ('Paprika-Bruscetta', 0.9065238),
 ('Paprikasoße', 0.9050808),
 ('Paprika-Lyoner', 0.9030753),
 ('Eingelegte Paprika', 0.9006094),
 ('Paprikasticks', 0.8994634),
 ('Paprika-Chili', 0.8991807),
 ('Paprikasuppe', 0.8970112),
 ('Chilischote', 0.89657897),
 ('Paprika Quark', 0.8945436),
 ('Geräucherte Paprika', 0.8937135),
 ('Paprika Snackis', 0.8935468),
 ('Gefüllte Paprika', 0.89313024)]

In [27]:
z = list(get_sentence_vectors([normalize_ingredient("Zucker")]))[0]
get_match(z, out_vecs, limit=30)

[('Zucker', 1.0),
 ('Gelier Zucker', 0.9193796),
 ('Gelier Zucker ', 0.9193796),
 ('Zuckerbrezel', 0.8992934),
 ('Fein Zucker', 0.88426435),
 ('Südzucker', 0.8819196),
 ('Zucker-Streusel', 0.88142365),
 ('Fruchtzucker', 0.87839866),
 ('Milchzucker', 0.8741473),
 ('Palmzucker', 0.8740871),
 ('Brauner Zucker', 0.8705967),
 ('Traubenzucker', 0.8682995),
 ('Traubenzucker ', 0.8682995),
 ('Gelierzucker', 0.8656156),
 ('Zuckermais', 0.8652842),
 ('Zuckerkarotten', 0.8642814),
 ('Feiner Zucker', 0.86407745),
 ('Zuckerschoten', 0.8623195),
 ('Backzucker', 0.8618116),
 ('Feinzucker', 0.8606885),
 ('Süßstoff', 0.86012936),
 ('Rohrzucker', 0.85917187),
 ('Zitronenzucker', 0.8589246),
 ('Süßstoff, Zucker', 0.8552715),
 ('Zucker Sticks', 0.85480726),
 ('Puderzucker', 0.8527151),
 ('Puderzucker ', 0.8527151),
 ('Zuckerstange', 0.8474887),
 ('Kandiszucker', 0.8458616),
 ('Zuckerfrei Bonbon', 0.8427986)]

In [28]:
z = list(get_sentence_vectors([normalize_ingredient("Knoblauch")]))[0]
get_match(z, out_vecs, limit=30)

[('Knoblauch', 0.99999994),
 ('Knoblauchgurken', 0.95750296),
 ('Knoblauchgurken ', 0.95750296),
 ('Knoblauchzehe', 0.9545716),
 ('Knoblauchspeck', 0.95325136),
 ('Knoblauchpüree', 0.9460995),
 ('Knoblauchschmalz', 0.9424915),
 ('Knoblauchecken', 0.9382149),
 ('Knoblauchsauce', 0.92497635),
 ('Knoblauchcreme', 0.9173736),
 ('Knoblauchcreme ', 0.9173736),
 ('Knoblauchbraten', 0.91436255),
 ('Knoblauch, getrocknet', 0.90933293),
 ('Knoblauchbutter', 0.908661),
 ('Knoblauchsoße', 0.90854347),
 ('Knoblauch Braten', 0.90431464),
 ('Knoblauch  sauce', 0.9030088),
 ('Knoblauch sauce', 0.9030088),
 ('Knoblauch-Baguette', 0.9024809),
 ('Knoblauchkick', 0.90135205),
 ('Knoblauchsalami', 0.90005994),
 ('Knoblauch-Creme', 0.8973919),
 ('Knoblauchsuppe', 0.8959118),
 ('Knoblauch Senf', 0.89517677),
 ('Knoblauch Pulver', 0.89178085),
 ('Knoblauchbeißer', 0.8914833),
 ('Knoblauch, frisch', 0.8905187),
 ('Knoblauchbaguette', 0.888514),
 ('Knoblauch Dressing', 0.88626623),
 ('Knoblauchecken, Knoblauch'

In [29]:
# i thought multiprocessing is impossible in jupyter notebook?
# better not question it
from multiprocessing import Pool

def jsonable(e):
    return [(a,float(b)) for a, b in e]

def get_match_w(t):
    ing, vec = t
    return ing, get_match(vec, out_vecs)

all_ings = {}
with Pool(8) as pool:
    for ing, match in pool.imap(get_match_w, tqdm(in_vecs), chunksize=100):
        all_ings[ing] = jsonable(match)

In [30]:

all_ings_orig = {ing: all_ings[normalize_ingredient(ing)] for ing in _in_names}

In [31]:
with open("data/recipe-ingredient-to-fooddb.json", "w") as f:
    json.dump(all_ings_orig, f)